In [3]:
# from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
PATH = "data/11.010-1-1.de.pdf"
def load_documents():
    # loader = DirectoryLoader(PATH, glob="*.md")
    loader = UnstructuredPDFLoader(PATH, language="de")
    documents = loader.load()
    return documents

In [5]:
def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

    return chunks

In [6]:
docs = load_documents()
chunks = split_text(docs)

c:\bjsChatBot\venv_bot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Split 1 documents into 164 chunks.
und kaum eindeutig vorzunehmen. Das Kirchengesetz zählt die «inneren» Angelegenheiten der Kirchen nicht abschliessend auf, sondern umschreibt sie in der Fassung vom 12. September 1995, in Kraft seit dem 1. Juli 1996, wie folgt: «Alles, was sich auf die Wortverkündigung, die Lehre, die Seelsorge,
{'source': 'data/11.010-1-1.de.pdf', 'start_index': 1573}


In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name=r"C:\bjsChatBot\distiluse-base-multilingual-cased-v2"
)
chunk_embeddings = [embeddings.embed_query(chunk.page_content) for chunk in chunks]


C:\Users\klara.orban\AppData\Local\Temp\ipykernel_12096\4009390109.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
No sentence-transformers model found with name C:\bjsChatBot\distiluse-base-multilingual-cased-v2. Creating a new one with mean pooling.


In [7]:
import re
def is_dense_text(text, min_word_ratio=0.7):
    words = text.split()
    num_words = len(words)
    num_alpha_words = sum(1 for w in words if re.search(r'[A-Za-zÄÖÜäöüß]', w))
    return num_words > 0 and (num_alpha_words / num_words) > min_word_ratio

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name = r"C:\bjsChatBot\multilingual-e5-base"
)
filter_chunks = [chunk for chunk in chunks if len(chunk.page_content.split()) > 20]
# chunk_embeddings = [embeddings.embed_query(chunk.page_content) for chunk in chunks]

dense = [c for c in filter_chunks if is_dense_text(c.page_content)]
chunk_embeddings = [embeddings.embed_query(chunk.page_content) for chunk in dense]


C:\Users\klara.orban\AppData\Local\Temp\ipykernel_24640\2334034970.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
No sentence-transformers model found with name C:\bjsChatBot\multilingual-e5-base. Creating a new one with mean pooling.


In [9]:
import faiss
import numpy as np

embedding_matrix = np.array(chunk_embeddings).astype("float32")

faiss.normalize_L2(embedding_matrix)

d = embedding_matrix.shape[1]

index = faiss.IndexFlatL2(d)  # L2 distance; for cosine, you can normalize first

index.add(embedding_matrix)

faiss.write_index(index, "chunk_index.faiss")

**Chroma Collection Save**

In [15]:
import chromadb
from chromadb.config import Settings
import os

# Point to the same folder where you stored Chroma DB
DIR = os.path.join(os.getcwd(), "chroma_db")
COLLECTION_NAME = "bjs_collection"

# Connect to the Chroma client
client = chromadb.Client(
    Settings(
        persist_directory=DIR,
        allow_reset=False  # do not erase existing data
    )
)

# Create or get your collection
collection = client.get_or_create_collection(name=COLLECTION_NAME)

# Add your chunks
document_to_add = [c.page_content for c in dense]
metadatas_to_add = [c.metadata for c in dense]
ids_to_add = [f"chunk_{i}" for i in range(len(dense))]
embeddings_to_add = chunk_embeddings

collection.add(
    documents=document_to_add,
    metadatas=metadatas_to_add,
    ids=ids_to_add,
    embeddings=embeddings_to_add
)

print("All chunks added to Chroma!")


All chunks added to Chroma!


In [17]:
docs = collection.get(include=["documents"])["documents"]
print(f"Documents in collection ({len(docs)}):")
for d in docs[6:10]:
    print(d)


Documents in collection (139):
2 Welches sind die «inneren» Angelegenheiten, in denen die Landeskirche autonom ist, und welches sind die «äusseren» Angelegenheiten, in denen sie nicht autonom ist? Die Abgrenzung ist nicht leicht und kaum eindeutig vorzunehmen. Das Kirchengesetz zählt die «inneren» Angelegenheiten der Kirchen
und kaum eindeutig vorzunehmen. Das Kirchengesetz zählt die «inneren» Angelegenheiten der Kirchen nicht abschliessend auf, sondern umschreibt sie in der Fassung vom 12. September 1995, in Kraft seit dem 1. Juli 1996, wie folgt: «Alles, was sich auf die Wortverkündigung, die Lehre, die Seelsorge,
dem 1. Juli 1996, wie folgt: «Alles, was sich auf die Wortverkündigung, die Lehre, die Seelsorge, den Kultus so- wie die religiöse Aufgabe der Landeskirchen, des Pfarramtes und der Kirch- gemeinden, die Diakonie und die Mission bezieht, gehört zu den inneren kirchlichen Angelegenheiten.» Zu den
die Diakonie und die Mission bezieht, gehört zu den inneren kirchlichen Angelege

In [18]:
query = "Kirche"
query_embedding = embeddings.embed_query(query)
query_embedding = np.array([query_embedding]).astype("float32")

results = collection.query(
    query_embeddings=query_embedding,
    n_results=50,
    include=["documents", "metadatas", "distances"]
)

docs = results["documents"][0]
metas = results["metadatas"][0]
dists = results.get("distances", [[None]*len(docs)])[0]

In [19]:
for i, (doc, meta, dist) in enumerate(zip(docs, metas, dists)):
    print(f"Rank: {i+1}")
    print(f"Distance: {dist}")
    print(doc)
    print("Metadata:", meta)
    print("-"*50)

top_data = [{"content": doc, "metadata": meta, "distance": dist}
            for doc, meta, dist in zip(docs, metas, dists)]

Rank: 1
Distance: 92.52220153808594
Geistes, der uns erweckt zur Gemeinde der Glaubenden und unter uns waltet als Geist der Kraft, der Liebe und der Zucht.
Metadata: {'start_index': 389, 'source': 'data/11.010-1-1.de.pdf'}
--------------------------------------------------
Rank: 2
Distance: 93.66752624511719
Gesuches in die Kirche aufgenommen worden sind alle Personen, die von auswärts in das bernische Kirchengebiet ein- gezogen sind und bisher schon einer dem Schweizerischen Evangeli- schen Kirchenbund angeschlossenen Kirche oder Gemeinschaft angehört haben die Glieder weiterer evangelischer Kirchen
Metadata: {'start_index': 8431, 'source': 'data/11.010-1-1.de.pdf'}
--------------------------------------------------
Rank: 3
Distance: 93.87762451171875
die Mitgliedschaft, die Errichtung und Ordnung der Kirchgemeinden sowie die Finanzordnung. Die Kirchen sind in diesem «äusseren» Bereich nicht au- tonom, auch wenn teilweise auch hier ihr Selbstverständnis tangiert ist. Viel- mehr beansp

In [20]:
import requests
import json

OLLAMA_URL = "http://localhost:11434/api/generate"

def ask_ollama(prompt, model="llama3.2:3b"):
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False  # False so we get the full reply in one response
    }
    response = requests.post(OLLAMA_URL, json=payload)
    response.raise_for_status()
    return response.json()["response"]

# if __name__ == "__main__":
#     answer = ask_ollama("Hallo! Erkläre mir, was ein Pfarrer macht.")
#     print("Answear:", answer)
inp = input("Ask: ")
print(ask_ollama(inp))



"Kirche" ist ein deutscher Begriff, der mehrere Bedeutungen haben kann. Hier sind einige mögliche Interpretationen:

1. **Kirche als religiöse Einrichtung**: Eine Kirche ist eine Gebäude oder ein Ort, an dem Menschen religiöse Gottesdienste und Rituale durchführen. Sie sind oft Teil eines christlichen oder anderen religiösen Systems.
2. **Kirche als historisches Gebäude**: Eine Kirche kann auch ein historisches Gebäude sein, das in der Vergangenheit als Kirche diente, aber heute möglicherweise anderen Zwecken dient, wie z.B. als Museum, Lagerhaus oder Wohngebäude.
3. **Kirche als soziale Institution**: In einem weiteren Sinne kann die Kirche auch eine soziale Institution sein, die sich um die Seelsorge und Unterstützung von Menschen kümmert, unabhängig von ihrer religiösen Zugehörigkeit.

In der deutschen Sprache gibt es auch einige Dialektbegriffe wie "Kirche" oder "Kirch", die unterschiedliche Bedeutungen haben können. Zum Beispiel bedeutet "Kirch" in einigen Regionen "Gebäude, das a

In [1]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
import requests

OLLAMA_URL = "http://localhost:11434/api/generate"

# Load existing Chroma DB
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma(persist_directory="./chroma_db", embedding_function=embedding)

def retrieve_context(query, k=3):
    docs = db.similarity_search(query, k=k)
    return "\n".join([d.page_content for d in docs])

def ask_ollama(prompt, model="llama3.2:3b"):
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }
    response = requests.post(OLLAMA_URL, json=payload)
    response.raise_for_status()
    return response.json()["response"]

if __name__ == "__main__":
    inp = input("Ask: ")
    context = retrieve_context(inp)
    final_prompt = f"Answer the question using the context below.\n\nContext:\n{context}\n\nQuestion: {inp}"
    print(ask_ollama(final_prompt))


C:\Users\klara.orban\AppData\Local\Temp\ipykernel_17880\3184641966.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\bjsChatBot\venv_bot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

**Query**

In [9]:
query = "Kirche"
query_embedding = embeddings.embed_query(query)
query_embedding = np.array([query_embedding]).astype("float32")

faiss.normalize_L2(query_embedding)

k = 50  # number of nearest neighbors
distances, indices = index.search(query_embedding, k)

# print(indices)
# print(distances)

ind_list = indices[0].tolist()
dist_list = distances[0].tolist()
print(ind_list)
print(dist_list)

[2, 134, 1, 10, 38, 22, 23, 29, 71, 9, 133, 48, 16, 49, 14, 123, 84, 136, 25, 40, 28, 15, 20, 80, 13, 8, 44, 70, 43, 74, 5, 11, 39, 120, 104, 41, 52, 12, 37, 67, 53, 124, 46, 42, 50, 4, 113, 78, 75, 122]
[0.36256182193756104, 0.36313238739967346, 0.37062764167785645, 0.37122902274131775, 0.3712293207645416, 0.3720112442970276, 0.37306541204452515, 0.3756100535392761, 0.37731778621673584, 0.37951019406318665, 0.37964537739753723, 0.38159143924713135, 0.38172584772109985, 0.38195109367370605, 0.3869449496269226, 0.3879121243953705, 0.38962823152542114, 0.3911231756210327, 0.3926393985748291, 0.39277225732803345, 0.39303216338157654, 0.39313286542892456, 0.3943527340888977, 0.39450979232788086, 0.39492180943489075, 0.3949991464614868, 0.3950178921222687, 0.39535078406333923, 0.39536190032958984, 0.39590728282928467, 0.39617717266082764, 0.3964671790599823, 0.397009015083313, 0.3979169726371765, 0.39843374490737915, 0.39860832691192627, 0.3995056450366974, 0.3997192978858948, 0.40216493606

In [10]:
# Suppose these are the results from FAISS
# indices = [116, 150, 117, 153, 128, 129, 130, 131, 120,  44]
# distances = [ 88.89868,   91.77553,   95.75629,   96.94322,   99.195854, 100.25284,
#   100.646454, 100.95151,  101.32193,  101.53297 ]

# Map indices back to your original text chunks and metadata
for idx, dist in zip(ind_list, dist_list):
    chunk = chunks[idx]  # get the Document object
    print(f"Index: {idx}, Distance: {dist}")
    print(chunk.page_content)
    print("Metadata:")
    print(chunk.metadata)
    print("-" * 50)


Index: 2, Distance: 0.36256182193756104
Geistes, der uns erweckt zur Gemeinde der Glaubenden und unter uns waltet als Geist der Kraft, der Liebe und der Zucht.
Metadata:
{'source': 'data/11.010-1-1.de.pdf', 'start_index': 389}
--------------------------------------------------
Index: 134, Distance: 0.36313238739967346
15

11.010

Reformierte Kirchen Bern-Jura-Solothurn

2 Sie bekennt sich damit zur Verpflichtung, für die Ausbreitung des Evangeli- ums in der weiten Welt einzustehen.

4 Der Geldhaushalt der Kirche

Art. B4

Lukas 16,10
Metadata:
{'source': 'data/11.010-1-1.de.pdf', 'start_index': 27331}
--------------------------------------------------
Index: 1, Distance: 0.37062764167785645
unseres Schöpfers und Vaters; Im Namen unseres Herrn und Heilandes Jesus Christus, der uns von Gott gemacht ist zur Weisheit, zur Gerechtigkeit, zur Heilung und zur Erlösung; Im Namen des Heiligen Geistes, der uns erweckt zur Gemeinde der Glaubenden und unter uns waltet als Geist der Kraft, der
Meta

**ChatBot**

In [ ]:
import requests

OLLAMA_URL = "http://localhost:11434/api/generate"

# Suppose these are your top FAISS results
retrieved_chunks = [
    {
        "content": "Geistes, der uns erweckt zur Gemeinde der Glaubenden...",
        "metadata": {"source": "data/11.010-1-1.de.pdf", "start_index": 389}
    },
    {
        "content": "11.010 Reformierte Kirchen Bern-Jura-Solothurn ... Art. B4 Lukas 16,10",
        "metadata": {"source": "data/11.010-1-1.de.pdf", "start_index": 27331}
    },
    {
        "content": "unseres Schöpfers und Vaters; Im Namen unseres Herrn...",
        "metadata": {"source": "data/11.010-1-1.de.pdf", "start_index": 24945}
    }
]

# Build the context string
context = "\n\n".join([f"{c['content']}\n(Source: {c['metadata']['source']})" for c in retrieved_chunks])

# Use a variable for the query
query = "Kirche"

# Build the prompt for Ollama
prompt = f"""
Beantworte die Frage basierend auf dem folgenden Kontext. 
Wenn die Antwort nicht im Kontext steht, sage 'Ich weiß es nicht'.

Kontext:
{context}

Frage: {query}
Antwort:
"""

# Send to Ollama
payload = {"model": "llama3.2:3b", "prompt": prompt, "stream": False}
response = requests.post(OLLAMA_URL, json=payload)
response.raise_for_status()
print("Ollama says:", response.json()["response"])


Ollama says: Kirche ist die Gemeinde der Glaubenden, erweckt durch den Geist.


In [ ]:
import requests
import json

OLLAMA_URL = "http://localhost:11434/api/generate"

def ask_ollama(prompt, model="llama3.2:3b"):
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False  # False so we get the full reply in one response
    }
    response = requests.post(OLLAMA_URL, json=payload)
    response.raise_for_status()
    return response.json()["response"]

if __name__ == "__main__":
    answer = ask_ollama("Hallo! Erkläre mir, was ein Pfarrer macht.")
    print("Ollama says:", answer)


In [1]:
import chromadb

CHROMA_DIR = r"chroma_db"
client = chromadb.PersistentClient(path=CHROMA_DIR)

collections = [c.name for c in client.list_collections()]
print("Collections:", collections)


Collections: ['bjs_colSmall', 'bjs_col']


In [2]:
collection = client.get_collection("bjs_colSmall")
print("Number of documents in collection:", collection.count())


Number of documents in collection: 173


In [3]:
results = collection.get(limit=5)  # get first 5 chunks
for i, doc in enumerate(results['documents']):
    print(f"Chunk {i}:\n{doc}\n{'-'*50}")


Chunk 0:
Reformierte Kirchen Bern-Jura-Solothurn 11.010
Verfassung der Evangelisch-reformierten 
Landeskirche des Kantons Bern
Vom 19. März 1946 (Stand 1. April 2003)
Im Namen und zur Ehre Gottes,
unseres Schöpfers und Vaters;
Im Namen unseres Herrn
und Heilandes Jesus Christus,
--------------------------------------------------
Chunk 1:
unseres Schöpfers und Vaters;
Im Namen unseres Herrn
und Heilandes Jesus Christus,
der uns von Gott gemacht ist zur Weisheit,
zur Gerechtigkeit, zur Heilung und zur
Erlösung;
Im Namen des Heiligen Geistes,
der uns erweckt zur Gemeinde der Glaubenden
und unter uns waltet als Geist
--------------------------------------------------
Chunk 2:
der uns erweckt zur Gemeinde der Glaubenden
und unter uns waltet als Geist
der Kraft, der Liebe und der Zucht.
Art.  P1 Einleitung
1 Wie schon die alte Staatsverfassung von 1893, so gewährleistet auch die  
Verfassung des Kantons Bern vom 6.  Juni 1993 den Landeskirchen aus -
------------------------------------------